In [2]:
from os.path import splitext
from os import listdir
import numpy as np
from glob import glob
import torch
from torch.utils.data import Dataset
import logging
from PIL import Image
import os
import json

class BasicDataset(Dataset):
    def __init__(self, imgs_dir, masks_dir, jsonDir, scale=1, mask_suffix='',val=False):
        self.imgs_dir = imgs_dir
        self.masks_dir = masks_dir
        self.jsonDir=jsonDir
        self.scale = scale
        self.mask_suffix = mask_suffix
        self.val=val
        assert 0 < scale <= 1, 'Scale must be between 0 and 1'
        print('initializing dataloader..')
        self.frameListUnfiltered = []
        imgsPerPathTemp=[]
        for r in imgs_dir:
            imgsPerPathTemp.append([f for f in [os.path.join(currentFolder, currentFile) for currentFolder, otherDirs, files in os.walk(os.path.expanduser(r)) for currentFile in files] if f[-8:]=='endo.png' or f[-4:]=='.jpg'])
        imgsPerPath=[]
        for path in imgsPerPathTemp:
            for img in path:
                self.frameListUnfiltered.append(img)
                
        print('matching images in json files to data')
        #make sure image in gallBladder annotation list
        self.frameList, self.coordsDict=self.matchImgFromJSON(self.jsonDir, self.frameListUnfiltered)
        #===        
        total=[os.path.basename(x) for x in self.frameList]
        unique=list(set(total))
        #==
        print('length total filepaths', len(total), 'length unique filenames', len(unique))
        logging.info(f'Creating dataset with {len(self.frameList)} examples')
    
    def __len__(self):
        #==return len(self.ids)
        return len(self.frameList)


    @classmethod
    def preprocess(cls, pil_img, scale, isMask=False):
        w, h = pil_img.size
        newW, newH = int(scale * w), int(scale * h)#==
        assert newW > 0 and newH > 0, 'Scale is too small'
        #pil_img = pil_img.resize((newW, newH))

        img_nd = np.array(pil_img)

        if len(img_nd.shape) == 2:
            img_nd = np.expand_dims(img_nd, axis=2)

        # HWC to CHW
        img_trans = img_nd.transpose((2, 0, 1))
 
        if img_trans.max() > 1 and not isMask:#don't want to normalize the mask values
            img_trans = img_trans / 255

        return img_trans


    def __getitem__(self, i):

        img_file = self.frameList[i] #==glob(self.imgs_dir + idx + '.*')
        
        coords=np.array(self.coordsDict[img_file]).squeeze()
        
        coords=np.concatenate((coords,np.array([[coords[0,0]],[coords[1,0]]])),axis=1)#to close contour
        
#         mask = Image.open(mask_file)#==
        img = Image.open(img_file)#==

        img = self.preprocess(img, self.scale)

        if self.val==True:

            return {
                'image': torch.from_numpy(img).type(torch.FloatTensor),
                'filename': img_file,
                'coords': coords
            }
        

        
        return {
            'image': torch.from_numpy(img).type(torch.FloatTensor),
            'filename': img_file,
            'coords': coords
        }
#==
    def make_ohe(self,mskOrig):
        """
        converts image with labels into the one-hot encoded format
        (img[...,] --> img[..., N_CLASSES])
        mode : `catId` or `trainId`
        """ 
        classDict={50:0, 11:1, 21:2, 13:3, 12:4, 31:5, 23:6, 24:7, 25:8, 32:9, 22:10, 33:11, 5:12, 255:0}
        classes = classDict.values()#13 classes including bg     
        
        ohe_labels = np.zeros((len(classes)-1,mskOrig.shape[1],mskOrig.shape[2]))#len(classes) -1 because 2 of them represent bg in our case
        for c in classes:
            ys, xs = np.where(mskOrig[0,...] == c)
            ohe_labels[c, ys, xs] = 1
        return ohe_labels.astype(int)
    
    def convertToSingle(self,RGBMask): 
        classDict={50:0, 11:1, 21:2, 13:3, 12:4, 31:5, 23:6, 24:7, 25:8, 32:9, 22:10, 33:11, 5:12, 255:0}
        for c in classDict.keys():
            RGBMask[0,np.where(RGBMask[0,:,:]==c)[0],np.where(RGBMask[0,:,:]==c)[1]]=classDict[c]#already converted to CHW so [0,:,:] #also in our dataset, all channels have the same value in the mask for easy processing
        RGBMask=RGBMask[0].reshape((1, RGBMask.shape[1],RGBMask.shape[2] ))
        if np.max(RGBMask)>12:
            RGBMask[np.where(RGBMask>12)]=0#just set outliers as background
            
        return RGBMask
    #find the location of image  with same size and name
    def findImgSize(self, imgFile,fullList, sizeCheck):
        for fullPath in fullList:
            if os.path.basename(fullPath)==imgFile:
                sizeObtained=os.path.getsize(fullPath)
                if sizeObtained==sizeCheck:
#                     print(print('size matched for ', imgFile, 'size in json ', sizeCheck, 'size in path ', os.path.getsize(fullPath)))
                    return [fullPath, sizeObtained]#found file
#                 else:
#                     print('size not matched for ', imgFile, 'size in json ', sizeCheck, 'size in path ', os.path.getsize(fullPath))
        
        return False #didn't find file

    
    def getfilesReq(self, coordsPerFile, pathsReq):

        filesReq=[]#store filenames with atleast one annotation

        #loop through all files per json and add all the ones which have any coordinates
        for path in pathsReq:      

            if len(coordsPerFile[path][0])>0:#make sure atleast one annotation
                filesReq.append(path)

        return filesReq    
    def checkNewFormat(self,perJSON,file,coordsPerFile,pathsReq):
        
        fileRead=perJSON[file]
        
        #manually mapped to video name
        fileToFolderDict={'NOGO_annotated09.json':'Vid24',
                         'via_project_21Apr2021_17h46m30s nogo 100.json':'video_01_00080',#short file
                         'via_project_24Apr2021_11h02m19s NOGO138.json':'video_01_00080',#short file
                         'via_project_26Apr2021_08h44m21s nogo159.json':'video_01_00080',#short file
                         'via_project_19Apr2021_12h38m19s nogo.json':'video_01_00080',
                         'NOGO1179via_project_23May2021_12h39m22s.json':'video_35_00940',
                         'via_project_25Apr2021_12h43m56snogo147.json':'video_01_00080',#short
                         'via_project_31Jul2021_22h01m03s.json':'Vid03',#can't seem to find a few files in this one
                         'NOGO8625via_project_16Jul2021_16h39m14s.json':'Vid24',
                         'NOGO14000via_project_06Jul2021_12h40m23s.json':'Vid24'#can't seem to find a few files in this one
                         }

        lsJsonInd=list(fileRead['file'].keys())#to find filename

        for index in range(len(lsJsonInd)):                        
            fileListed=fileRead['file'][lsJsonInd[index]]['fname']#file listed in json
            metadataKeys=list(fileRead['metadata'].keys())#to find coordinates
            vertices=[]#start xy coordinate list
            if len(fileRead['metadata'][metadataKeys[index]]['xy'])>1:#make sure something is there in coords
                rawPoints=fileRead['metadata'][metadataKeys[index]]['xy'][1:]
                prev=0#starting index for coords

                for i in range(2,len(rawPoints),2):#starts from 2 so we start from prev=0 and i=2
                    vertices.append(rawPoints[prev:i])#make list of x,y coordinates
                    prev=i

                if fileListed.find('_no-go')!=-1:
                    fileName=fileListed[:fileListed.find('_no-go')]+'.jpg'#take out "no-go" part if exists and make it a regular filename
                else:
                    fileName=fileListed
                if 'endo' in fileName:#toggle between 2 folders found
                    pathReq=os.path.join(os.getcwd(),'archive',fileToFolderDict[file]+'/'+fileName)
                else:                
                    pathReq=os.path.join(os.getcwd(),'CTC_Frames',fileToFolderDict[file]+'/'+fileName)                    

                if os.path.exists(pathReq):
                    coordsPerFile[pathReq]=vertices#store list of x,y coordinates
                    pathsReq.append(pathReq)
                else:
                    print('DNE: ', pathReq)

        return coordsPerFile, pathsReq
        
        
    def getCoords(self, perJSON, frameList):
        coordsPerFile=dict()#store coordinate annotations (polygon vertices) with filename as key
        fileCount=0
        notFound=0
        pathsReq=[]
        dups=0
        for file in list(perJSON.keys()):     #loop through json file data
#             print('checking for coordinates in file name:', file)
            if '_via_img_metadata' not in perJSON[file].keys():
                print('skipping new json format: ', file)
                #coordsPerFile, pathsReq=self.checkNewFormat(perJSON,file,coordsPerFile,pathsReq)
                continue
            for key2 in perJSON[file]['_via_img_metadata'].keys():  
                sizeReq=perJSON[file]['_via_img_metadata'][key2]['size']#for comparison to find proper path in "findImgSize"
                fName=perJSON[file]['_via_img_metadata'][key2]['filename']#for comparison to find proper path in "findImgSize"
                pathReq=self.findImgSize(fName, frameList, sizeReq)#find the location of image  with same size and name
                
       
                if pathReq==False:
                    if notFound<2 and '_' not in fName:
                        print('file not found', fName, sizeReq)
                    if '_' in fName:
                        print('file not found', fName, sizeReq)
                        
                    notFound+=1
                    continue
                else:
                    fullPath=pathReq[0]
                    if fullPath in pathsReq:#maybe has more than 1 region
                        dups+=1                        
#                         print('dup', pathReq[0])
                    else:
                        pathsReq.append(fullPath)
                
                coordsPerFile[fullPath]=[] #initialize as list            
                xs=None
                ys=None
                if len(perJSON[file]['_via_img_metadata'][key2]['regions'])==0: #no coords found
                    coordsPerFile[fullPath].append([])#just to note there are no annotations                      
                    continue            
                #store x, y coords for polygon vertices
                xs=perJSON[file]['_via_img_metadata'][key2]['regions'][0]['shape_attributes']['all_points_x']
                ys=perJSON[file]['_via_img_metadata'][key2]['regions'][0]['shape_attributes']['all_points_y']
                coordsPerFile[fullPath].append([xs,ys]) #store x,y per file                            
                            
        print('files not found coords, multiple regions ', notFound, dups) 
        return coordsPerFile, pathsReq

    def matchImgFromJSON(self, dirReq, frameList):
        #====read json, get coords for each file, get files where atleast one set of coords specified====
        perJSON=dict()    
        for file in os.listdir(dirReq):
            with open(dirReq+file) as f:
                perJSON[file] = json.load(f) #get json list
        fileCount=0
        coordsPerFile, pathsReq=self.getCoords(perJSON, frameList)
        print('initial amount of files:', len(list(coordsPerFile.keys())), len(pathsReq), len(list(set(pathsReq))))
#         pathsReq=list(set(pathsReq))
        filesReq=self.getfilesReq(coordsPerFile, pathsReq)        
#         filesReq=pathsReq#maybe there should be coordinates without a segment
        
        print('after filesReq (filters for at least one annotation)', len(filesReq))
        #======================================================

        #===store filename and size, find exact path of file in dataset subfolders and put in a list===

        #===================================================================================
        return filesReq, coordsPerFile

#==    
        
class cholec8KDatasetGallBladderMasks(BasicDataset):#==
    def __init__(self, imgs_dir, masks_dir, jsonDir, scale=1):
        super().__init__(imgs_dir, masks_dir, jsonDir, scale, mask_suffix='_mask')



In [9]:
import sys
import torch
import torch.nn.functional as F
from tqdm import tqdm
#from utils.datasetGB import BasicDataset
#from dice_loss import dice_coeff
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
    
#================
# needs mask in the mask directory with the same name as frame
#================
device='cuda'#could use 'cpu' if needed
root_path = os.getcwd() # remember to verify during actual training#==
dirReqImg=[root_path+'/archive/',
           root_path+'/CTC_Frames/']

dirReqMsk=root_path+'/gallBladder/masks/'
dirReqJson=root_path+'/GallbladderFiles/'#might not be needed for this

DS=cholec8KDatasetGallBladderMasks(dirReqImg, dirReqMsk, dirReqJson)

# Getting training and testing files

In [10]:
reqTrain=[]
reqTest=[]
foldersTest=['CTC_Frames', 'video24']
foldersTrain=['video01', 'video18', 'video20', 'video35', 'video55']
for f in DS:    
    folder=os.path.basename(os.path.dirname(os.path.dirname(f['filename'])))
    if folder in foldersTest:
        reqTest.append(f['filename'])
    elif folder in foldersTrain:
        reqTrain.append(f['filename'])
        
print('no of training files', len(reqTrain), 'no of testing files', len(reqTest))


# Putting dataloader information into Detectron2-required format

In [4]:
from detectron2.structures import BoxMode#remember to change to  "from detectron2.structures import BoxMode" when doing detectron
from detectron2.data import MetadataCatalog, DatasetCatalog

dataset_dicts_train = []
dataset_dicts_val = []
c=0

for idx, loaded in enumerate(DS):    
    record = {}    
    height, width = loaded['image'][0].shape
    
    record["file_name"] = loaded['filename']
    record["image_id"] = idx
    record["height"] = height
    record["width"] = width    
    objs = []
#==no for loop because seems to be only one zone for gallbladder==

    px = loaded["coords"][0]
    py = loaded["coords"][1]
    poly = [(x + 0.5, y + 0.5) for x, y in zip(px, py)]
    poly = [p for x in poly for p in x]

    obj = {
        "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
        "bbox_mode": BoxMode.XYXY_ABS,
        "segmentation": [poly],
        "category_id": 0,
    }

    objs.append(obj)
    
    record["annotations"] = objs
    
    if loaded['filename'] in reqTrain:#splitting training and testing
        dataset_dicts_train.append(record)
    else:
        dataset_dicts_val.append(record)
# print(c)

In [5]:
# dataset_dicts_train[0]

# == Augmenting dataset below (commented out) ====

In [6]:
# from detectron2.data import transforms as T
# import cv2
# index=500
# print('Are you sure you want to continue? Enter y if yes. If not enter anything else. ')
# x=input()

    
# augs = T.AugmentationList([
#     T.RandomBrightness(0.9, 1.1),
#     T.RandomFlip(prob=0.5),
#     T.RandomCrop("absolute", (640, 640))
# ])  # type: T.Augmentation

# coordDict=dict()
# if x=='y':    
#     for d in dataset_dicts_train:
#     # Define a sequence of augmentations:

#         img=np.array(Image.open(d['file_name']))#np.array(DS[index]['image'].permute(1,2,0))
#         sem_seg=np.array(d['annotations'][0]['segmentation'])
#         boxes=np.array(d['annotations'][0]['bbox'])

#         # Define the augmentation input ("image" required, others optional):
#         toTransform = T.AugInput(img, boxes=boxes, sem_seg=sem_seg)
#         # Apply the augmentation:
#         transform = augs(toTransform)  # type: T.Transform
#         image_transformed = toTransform.image  # new image
#         sem_seg_transformed = toTransform.sem_seg  # new semantic segmentation
#         box_transformed = toTransform.boxes

#         reverse=[]
#         ii=0
#         temp=[]
#         for x in sem_seg[0]:    
#             if ii%2==0 and ii!=0:        
#                 reverse.append(temp)
#                 temp=[]

#             temp.append(int(x))
#             ii+=1

#         # For any extra data that needs to be augmented together, use transform, e.g.:
#         # image2_transformed = transform.apply_image(image2)
#         polygons_transformed = transform.apply_polygons([np.array(reverse)])    

#         newLocation=os.getcwd()+'/augmentedGB/images/Aug_'+os.path.basename(d['file_name'])
#         toSave = Image.fromarray(np.uint8(image_transformed))
#         toSave.save(newLocation)

#         coordDict[newLocation]=dict()
#         coordDict[newLocation]['oldLocation']=d['file_name']
#         coordDict[newLocation]['bbox']=box_transformed
#         coordDict[newLocation]['polygon']=polygons_transformed
#         coordDict[newLocation]['shape']=image_transformed.shape    
# else:
#     print('Operation cancelled')
    
# #     toPlot=cv2.fillPoly(np.array(image_transformed), pts = [np.array(polygons_transformed,dtype=np.int32)], color =(255,255,255))
# #     x,y,w,h = box_transformed[0]
# #     toPlot=cv2.rectangle(toPlot,(x,y),(x+w,y+h),(0,255,0),2)
# #     plt.imshow(toPlot)
# #     break

In [7]:
# import copy

# coordTest=copy.deepcopy(coordDict)

# coordTest
# def convertToPythonInteger(dictReq):
#     individual=[float(n) for nn in dictReq for n in nn]
#     toList=[]
#     c=[]    
#     for i, xy in enumerate(individual):
#         if i%2==0 and i>0:
#             toList.append(c)
#             c=[]
#         c.append(xy)    
#     return toList
      
# coordDict=copy.deepcopy(coordTest)

# for file in coordDict.keys():    
#     if len(coordDict[file]['bbox'])>0 and len(coordDict[file]['polygon'])>0:
#         coordDict[file]['bbox']=list(coordDict[file]['bbox'][0])
#         coordDict[file]['polygon']=list(coordDict[file]['polygon'][0])
#         coordDict[file]['bbox']=[int(n) for n in coordDict[file]['bbox']]
#         coordDict[file]['polygon']=convertToPythonInteger(coordDict[file]['polygon'])


In [8]:
# coordDict[file]['polygon'][0], type(coordDict[file]['bbox'])
# coordDictFinal=copy.deepcopy(coordDict)
# coordDict=copy.deepcopy(coordDictFinal)
# files=list(coordDictFinal.keys())
# for file in files:    
#     if len(coordDict[file]['bbox'])==0 or len(coordDict[file]['polygon'])==0:
#         del coordDict[file]

In [9]:
# json_object = json.dumps(coordDict, indent = 4)

# # Writing to sample.json
# with open("augCoords.json", "w") as outfile:
#     outfile.write(json_object)

# Adding augmented coordinates to our training dataset (augmented coordinates were stored in the json file in the commented code above)

In [10]:
import json

def readJson(filename):
    # Writing to sample.json
    with open(filename, "r") as inFile:
        fileRead = json.load(inFile)
    return fileRead
def XyxyPoly(polyCoords):
    xyxy=[xy for pair in polyCoords for xy in pair]
    return xyxy
    
aug=readJson("augCoords.json")
totalLength=len(DS)
for idx, file in enumerate(list(aug.keys())):
    recordAug={}
    poly=XyxyPoly(aug[file]['polygon'])
    recordAug['file_name']=file
    recordAug["image_id"] = idx+totalLength
    recordAug["height"] = aug[file]['shape'][0]
    recordAug["width"] = aug[file]['shape'][1]
    objs = []
    obj={
        "bbox": aug[file]['bbox'],
        "bbox_mode": BoxMode.XYXY_ABS,
        "segmentation": [poly],
        "category_id": 0,        
    }
    objs.append(obj)
    
    recordAug['annotations']=objs
    
    dataset_dicts_train.append(recordAug)

# aug['/scratch/ssd002/home/skhalid/cv_vector/augmentedGB/images/Aug_frame_400_endo.png']['shape']

In [11]:
#adding first 2 coordinates to close contour
for d in dataset_dicts_train:
    if d['annotations'][0]['segmentation'][0][-2:]!=d['annotations'][0]['segmentation'][0][:2]:
        print('closing polygon contour for ', d['file_name'])
        d['annotations'][0]['segmentation'][0].append(d['annotations'][0]['segmentation'][0][0])#adding first 2 coordinates to close contour
        d['annotations'][0]['segmentation'][0].append(d['annotations'][0]['segmentation'][0][1])    



appending  /scratch/ssd002/home/skhalid/cv_vector/augmentedGB/images/Aug_frame_400_endo.png
appending  /scratch/ssd002/home/skhalid/cv_vector/augmentedGB/images/Aug_frame_401_endo.png
appending  /scratch/ssd002/home/skhalid/cv_vector/augmentedGB/images/Aug_frame_402_endo.png
appending  /scratch/ssd002/home/skhalid/cv_vector/augmentedGB/images/Aug_frame_403_endo.png
appending  /scratch/ssd002/home/skhalid/cv_vector/augmentedGB/images/Aug_frame_404_endo.png
appending  /scratch/ssd002/home/skhalid/cv_vector/augmentedGB/images/Aug_frame_405_endo.png
appending  /scratch/ssd002/home/skhalid/cv_vector/augmentedGB/images/Aug_frame_406_endo.png
appending  /scratch/ssd002/home/skhalid/cv_vector/augmentedGB/images/Aug_frame_407_endo.png
appending  /scratch/ssd002/home/skhalid/cv_vector/augmentedGB/images/Aug_frame_408_endo.png
appending  /scratch/ssd002/home/skhalid/cv_vector/augmentedGB/images/Aug_frame_409_endo.png
appending  /scratch/ssd002/home/skhalid/cv_vector/augmentedGB/images/Aug_frame_4

# Checking augmentation coordinates by plotting(commented out)

In [12]:
# # fileReq='/scratch/ssd002/home/skhalid/cv_vector/augmentedGB/images/Aug_frame_400_endo.png'
# allFiles=list(aug.keys())
# fileReq=allFiles[40]
# pp=aug[fileReq]['polygon']
# pp, [xy for pair in pp for xy in pair]
# imgReq=Image.open(fileReq)
# toShow=cv2.fillPoly(img, pts = [np.array(pp,dtype=np.int32)], color =(255,255,255))

# plt.imshow(toShow)

In [13]:
print('length of training dataset after augmentation', len(dataset_dicts_train))

length of training dataset after augmentation 1797


# == Augmentation section above ====

In [14]:
# sem_seg
# toPlot

In [15]:
# polygons_transformed,reverse

In [16]:
#plt.imshow(image_transformed)
# img.shape,image_transformed.shape,image_transformed.dtype,img.dtype

In [17]:
# #===========CHECKING ACTUAL LABELS THROUGH GRAPH============
# import cv2

# for index in range(1100,1110):
#     file=dataset_dicts_train[index]['file_name'],
#     coords=dataset_dicts_train[index]['annotations'][0]['segmentation']
#     v=[]
#     temp=[]
#     for c in coords:
#         #print(len(c))
#         for i, cc in enumerate(c):                
#             if i%2==0:
#                 v.append(temp)
#                 temp=[]

#             temp.append(cc)

#     # file,v,dataset_dicts_train[0]
#     #print(file[0],dataset_dicts_train[0])
#     img=np.array(Image.open(file[0]))
#     plt.figure(figsize=(15,15))
#     plt.imshow(img)
#     img=cv2.fillPoly(img, pts = [np.array(v[1:],dtype=np.int32)], color =(255,255,255))
#     plt.figure(figsize=(15,15))
#     plt.imshow(img)

# Registering different characteristics of the dataset into the Detectron2 Registry

In [18]:
DatasetCatalog.register("bladder_train", lambda d="train": dataset_dicts_train)
DatasetCatalog.register("bladder_val", lambda d="val": dataset_dicts_val)

MetadataCatalog.get("bladder_train").set(thing_classes=["no_go_zone"])
MetadataCatalog.get("bladder_val").set(thing_classes=["no_go_zone"])

MetadataCatalog.get("bladder_train").stuff_colors = [(0,128,128)]
MetadataCatalog.get("bladder_val").stuff_colors = [(0,128,128)]

MetadataCatalog.get("bladder_train").stuff_colors = [(0,128,128)]
MetadataCatalog.get("bladder_val").stuff_colors = [(0,128,128)]

MetadataCatalog.get("bladder_train").thing_colors = [(0,128,128)]
MetadataCatalog.get("bladder_val").thing_colors = [(0,128,128)]

MetadataCatalog.get("bladder_train").set(stuff_classes=["no_go_zone"])
MetadataCatalog.get("bladder_val").set(stuff_classes=["no_go_zone"])


Metadata(name='bladder_val', stuff_classes=['no_go_zone'], stuff_colors=[(0, 128, 128)], thing_classes=['no_go_zone'], thing_colors=[(0, 128, 128)])

In [19]:
bladder_metadata = MetadataCatalog.get("bladder_val")

# Custom classes for implementing automatic evaluation while training our newly-registered dataset 

In [20]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2 import model_zoo

from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

class MyTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        return COCOEvaluator(dataset_name, cfg, True, output_folder)
                     
    def build_hooks(self):
        hooks = super().build_hooks()
        hooks.insert(-1,LossEvalHook(
            cfg.TEST.EVAL_PERIOD,
            self.model,
            build_detection_test_loader(
                self.cfg,
                self.cfg.DATASETS.TEST[0],
                DatasetMapper(self.cfg,True)
            )
        ))
        return hooks

In [21]:
from detectron2.engine.hooks import HookBase
from detectron2.evaluation import inference_context
from detectron2.utils.logger import log_every_n_seconds
from detectron2.data import DatasetMapper, build_detection_test_loader
import detectron2.utils.comm as comm
import torch
import time
import datetime

class LossEvalHook(HookBase):
    def __init__(self, eval_period, model, data_loader):
        self._model = model
        self._period = eval_period
        self._data_loader = data_loader
    
    def _do_loss_eval(self):
        # Copying inference_on_dataset from evaluator.py
        total = len(self._data_loader)
        num_warmup = min(5, total - 1)
            
        start_time = time.perf_counter()
        total_compute_time = 0
        losses = []
        for idx, inputs in enumerate(self._data_loader):            
            if idx == num_warmup:
                start_time = time.perf_counter()
                total_compute_time = 0
            start_compute_time = time.perf_counter()
            if torch.cuda.is_available():
                torch.cuda.synchronize()
            total_compute_time += time.perf_counter() - start_compute_time
            iters_after_start = idx + 1 - num_warmup * int(idx >= num_warmup)
            seconds_per_img = total_compute_time / iters_after_start
            if idx >= num_warmup * 2 or seconds_per_img > 5:
                total_seconds_per_img = (time.perf_counter() - start_time) / iters_after_start
                eta = datetime.timedelta(seconds=int(total_seconds_per_img * (total - idx - 1)))
                log_every_n_seconds(
                    logging.INFO,
                    "Loss on Validation  done {}/{}. {:.4f} s / img. ETA={}".format(
                        idx + 1, total, seconds_per_img, str(eta)
                    ),
                    n=5,
                )
            loss_batch = self._get_loss(inputs)
            losses.append(loss_batch)
        mean_loss = np.mean(losses)
        self.trainer.storage.put_scalar('validation_loss', mean_loss)
        comm.synchronize()

        return losses
            
    def _get_loss(self, data):
        # How loss is calculated on train_loop 
        metrics_dict = self._model(data)
        metrics_dict = {
            k: v.detach().cpu().item() if isinstance(v, torch.Tensor) else float(v)
            for k, v in metrics_dict.items()
        }
        total_losses_reduced = sum(loss for loss in metrics_dict.values())
        return total_losses_reduced
        
        
    def after_step(self):
        next_iter = self.trainer.iter + 1
        is_final = next_iter == self.trainer.max_iter
        if is_final or (self._period > 0 and next_iter % self._period == 0):
            self._do_loss_eval()
        self.trainer.storage.put_scalars(timetest=12)


# Starting training

In [23]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("bladder_train",)
cfg.DATASETS.TEST = ("bladder_val",)
cfg.TEST.EVAL_PERIOD = 200
cfg.SOLVER.CLIP_GRADIENTS.ENABLED=True

cfg.MODEL.SEM_SEG_HEAD.IGNORE_VALUE = -1 # was set to 255
cfg.OUTPUT_DIR='./output/jupyterModel/'
cfg.SOLVER.CHECKPOINT_PERIOD = 2000

cfg.TEST.EVAL_PERIOD = 200
cfg.DATALOADER.NUM_WORKERS = 8
cfg.SOLVER.WEIGHT_DECAY = 0.00001 #trying this out for generalization issues
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2 #number of images for 1 iteration to be complete
cfg.SOLVER.WARMUP_ITERS = 0
cfg.SOLVER.BASE_LR = 0.000025  # pick a good LR
cfg.SOLVER.MAX_ITER = 500 # for demonstration purposes
cfg.SOLVER.STEPS = []    # decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.

evaluator = COCOEvaluator(dataset_name="bladder_val",output_dir=os.getcwd()+"/jsonOutput")
val_loader = build_detection_test_loader(cfg, "bladder_val")

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = MyTrainer(cfg) 

trainer.resume_or_load(resume=False)
trainer.train()

[11/15 18:54:46 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[11/15 18:54:46 d2.data.common]: Serializing 111 elements to byte tensors and concatenating them all ...
[11/15 18:54:46 d2.data.common]: Serialized dataset takes 0.15 MiB
[11/15 18:54:46 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[11/15 18:54:46 d2.data.common]: Serializing 111 elements to byte tensors and concatenating them all ...
[11/15 18:54:46 d2.data.common]: Serialized dataset takes 0.15 MiB
[11/15 18:54:48 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), st

[11/15 18:54:48 d2.data.build]: Removed 0 images with no usable annotations. 1797 images left.
[11/15 18:54:48 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in training: [ResizeShortestEdge(short_edge_length=(640, 672, 704, 736, 768, 800), max_size=1333, sample_style='choice'), RandomFlip()]
[11/15 18:54:48 d2.data.build]: Using training sampler TrainingSampler
[11/15 18:54:48 d2.data.common]: Serializing 1797 elements to byte tensors and concatenating them all ...
[11/15 18:54:48 d2.data.common]: Serialized dataset takes 1.43 MiB
[11/15 18:54:49 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in training: [ResizeShortestEdge(short_edge_length=(640, 672, 704, 736, 768, 800), max_size=1333, sample_style='choice'), RandomFlip()]
[11/15 18:54:49 d2.data.common]: Serializing 111 elements to byte tensors and concatenating them all ...
[11/15 18:54:49 d2.data.common]: Serialized dataset takes 0.15 MiB


roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}
roi_heads.mask_head.predictor.{bias, weight}


[11/15 18:54:49 d2.engine.train_loop]: Starting training from iteration 0
[11/15 18:54:57 d2.utils.events]:  eta: 0:03:10  iter: 19  total_loss: 1.439  loss_cls: 0.5867  loss_box_reg: 0.1257  loss_mask: 0.6897  loss_rpn_cls: 0.02108  loss_rpn_loc: 0.01048  time: 0.3840  data_time: 0.0342  lr: 2.5e-05  max_mem: 9547M
[11/15 18:55:05 d2.utils.events]:  eta: 0:03:01  iter: 39  total_loss: 1.151  loss_cls: 0.3057  loss_box_reg: 0.1234  loss_mask: 0.6819  loss_rpn_cls: 0.02151  loss_rpn_loc: 0.007314  time: 0.3792  data_time: 0.0037  lr: 2.5e-05  max_mem: 9637M
[11/15 18:55:13 d2.utils.events]:  eta: 0:02:41  iter: 59  total_loss: 1.015  loss_cls: 0.182  loss_box_reg: 0.1032  loss_mask: 0.6756  loss_rpn_cls: 0.03317  loss_rpn_loc: 0.008113  time: 0.3744  data_time: 0.0036  lr: 2.5e-05  max_mem: 9637M
[11/15 18:55:20 d2.utils.events]:  eta: 0:02:33  iter: 79  total_loss: 0.9731  loss_cls: 0.1522  loss_box_reg: 0.09934  loss_mask: 0.6617  loss_rpn_cls: 0.04934  loss_rpn_loc: 0.008666  time: 0

[11/15 18:56:26 d2.evaluation.testing]: copypaste: Task: segm
[11/15 18:56:26 d2.evaluation.testing]: copypaste: AP,AP50,AP75,APs,APm,APl
[11/15 18:56:26 d2.evaluation.testing]: copypaste: 4.5769,13.8817,2.0365,nan,0.1028,5.0044
[11/15 18:56:27 detectron2]: Loss on Validation  done 11/111. 0.0000 s / img. ETA=0:00:07
[11/15 18:56:32 detectron2]: Loss on Validation  done 67/111. 0.0000 s / img. ETA=0:00:03
[11/15 18:56:37 d2.utils.events]:  eta: 0:01:56  iter: 199  total_loss: 0.8685  loss_cls: 0.1011  loss_box_reg: 0.08836  loss_mask: 0.6084  loss_rpn_cls: 0.03479  loss_rpn_loc: 0.00698  validation_loss: 0.8438  time: 0.3755  data_time: 0.0036  lr: 2.5e-05  max_mem: 9637M
[11/15 18:56:44 d2.utils.events]:  eta: 0:01:49  iter: 219  total_loss: 0.8563  loss_cls: 0.1215  loss_box_reg: 0.1312  loss_mask: 0.5811  loss_rpn_cls: 0.01889  loss_rpn_loc: 0.007163  validation_loss: 0.8438  time: 0.3769  data_time: 0.0035  lr: 2.5e-05  max_mem: 9637M
[11/15 18:56:53 d2.utils.events]:  eta: 0:01:42

[11/15 18:58:10 d2.evaluation.coco_evaluation]: Some metrics cannot be computed and is shown as NaN.
[11/15 18:58:10 d2.engine.defaults]: Evaluation results for bladder_val in csv format:
[11/15 18:58:10 d2.evaluation.testing]: copypaste: Task: bbox
[11/15 18:58:10 d2.evaluation.testing]: copypaste: AP,AP50,AP75,APs,APm,APl
[11/15 18:58:10 d2.evaluation.testing]: copypaste: 15.6688,41.8235,9.2680,nan,0.9364,16.3504
[11/15 18:58:10 d2.evaluation.testing]: copypaste: Task: segm
[11/15 18:58:10 d2.evaluation.testing]: copypaste: AP,AP50,AP75,APs,APm,APl
[11/15 18:58:10 d2.evaluation.testing]: copypaste: 17.4052,47.0916,6.1416,nan,0.1890,18.2673
[11/15 18:58:11 detectron2]: Loss on Validation  done 11/111. 0.0001 s / img. ETA=0:00:07
[11/15 18:58:16 detectron2]: Loss on Validation  done 68/111. 0.0000 s / img. ETA=0:00:03
[11/15 18:58:21 d2.utils.events]:  eta: 0:00:38  iter: 399  total_loss: 0.7543  loss_cls: 0.1055  loss_box_reg: 0.1146  loss_mask: 0.5216  loss_rpn_cls: 0.01834  loss_rpn

[11/15 18:59:27 d2.evaluation.coco_evaluation]: Some metrics cannot be computed and is shown as NaN.
[11/15 18:59:27 d2.engine.defaults]: Evaluation results for bladder_val in csv format:
[11/15 18:59:27 d2.evaluation.testing]: copypaste: Task: bbox
[11/15 18:59:27 d2.evaluation.testing]: copypaste: AP,AP50,AP75,APs,APm,APl
[11/15 18:59:27 d2.evaluation.testing]: copypaste: 17.3277,50.7387,4.4452,nan,0.5687,18.1150
[11/15 18:59:27 d2.evaluation.testing]: copypaste: Task: segm
[11/15 18:59:27 d2.evaluation.testing]: copypaste: AP,AP50,AP75,APs,APm,APl
[11/15 18:59:27 d2.evaluation.testing]: copypaste: 22.4055,57.5678,6.8927,nan,0.1219,23.4370


# Setting up configuration and instantiating a predictor

In [24]:
from detectron2.engine import DefaultPredictor

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("bladder_train",)
cfg.DATASETS.TEST = ("bladder_val",)
cfg.TEST.EVAL_PERIOD = 200
cfg.DATALOADER.NUM_WORKERS = 8
cfg.SOLVER.WEIGHT_DECAY = 0.00001 #trying this out for generalization
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2 #number of images for 1 iteration to be complete
cfg.SOLVER.WARMUP_ITERS = 0
cfg.SOLVER.BASE_LR = 0.00005  # was 0.000025 # pick a good LR
cfg.SOLVER.MAX_ITER = int(len(dataset_dicts_train)/cfg.SOLVER.IMS_PER_BATCH)*5 #(last number in the eqn is epochs)  #iterations for one epoch * num epochs
# cfg.SOLVER.STEPS = [300, 600]    # for decaying learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
cfg.SOLVER.CLIP_GRADIENTS.ENABLED=True
cfg.MODEL.SEM_SEG_HEAD.IGNORE_VALUE = -1 # was set to 255
cfg.OUTPUT_DIR='./output/jupyterModel/'

# model_0009999-Copy1.pth
# model_final-1e_5_23Aug.pth

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.2   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

# Plotting results 

In [12]:
from PIL import Image
from detectron2.utils.visualizer import Visualizer

import matplotlib.pyplot as plt

count=0
for i in range(len(reqTest)):

        pathReq=reqTest[i]

        im = np.array(Image.open(pathReq))
        v = Visualizer(im[:, :, ::-1],
                       metadata=bladder_metadata, 
                       scale=0.5                       
        )

        outputs = predictor(im)
        inst_qty = len((outputs['instances'])._fields['pred_boxes'])

        if inst_qty:
            out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
            plt.figure(figsize=(15,15))
            plt.imshow(out.get_image()[:, :, ::-1])
            count+=1